In [0]:
# Notebook: 3_silver_to_gold

silver_prefix = "/Volumes/movies_catalog/silver/silver_zone_movies"
gold_prefix = "/Volumes/movies_catalog/gold/gold_zone_movies"

df = spark.read.format("delta").load(f"{silver_prefix}/top100_combined")

from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window


window = Window.partitionBy("Year").orderBy(col("IMDb_Rating").desc())

top10 = (
    df.withColumn("rn", row_number().over(window))
      .filter(col("rn") <= 10)
      .drop("rn")
)

top10.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(f"{gold_prefix}/top10_by_year")

print("gold written ✅")
